# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## 1. I'll compute the camera calibration using chessboard images

Find the object points and image points for the calibration

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('/Users/jianshuwang/Desktop/CarND-Advanced-Lane-Lines/camera_cal/calibration*.jpg')


# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## 2. Apply Distortion Correction to Raw Images

Now we have objpoints and imgpoints of the camera and we will use it for the next step, calibration and undistortion


In [ ]:
import matplotlib.image as mpimg

def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    undist = np.copy(dst)  # Delete this line
    return undist

img = cv2.imread('/Users/jianshuwang/Desktop/CarND-Advanced-Lane-Lines/test_images/my_test10.jpg')
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

undistorted = cal_undistort(img, objpoints, imgpoints)

%matplotlib inline
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/undistort_output.png')


## 3. Use Color Transforms, Gradients, etc., to create a thresholded binary image.

Now we are going to get the gradient in x axis and y axis and combine with HLS 

In [ ]:
def combine_select(img, Sthresh=(0, 255), mag_thresh=(0, 255), sobel_kernel=3):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    #H = hls[:,:,0]
    #L = hls[:,:,1]
    S = hls[:,:,2]
    # 2) Apply a threshold to the S channel
    binary_S = np.zeros_like(S)
    binary_S[(S > Sthresh[0]) & (S <= Sthresh[1])] = 1
    
    
    Lthresh = (220,255)
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    L = luv[:,:,0]
    binary_L = np.zeros_like(L)
    binary_L[(L > Lthresh[0]) & (L <= Lthresh[1])] = 1
    
    Bthresh = (135,200)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    B = lab[:,:,2]
    binary_B = np.zeros_like(B)
    binary_B[(B > Bthresh[0]) & (B <= Bthresh[1])] = 1
    
    
    
    
    # 3) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 4) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 5) Calculate the magnitude 
    sobel = np.sqrt(sobelx**2 + sobely**2)
    #sobel = sobelx
    #sobel = sobely
    # 6) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*sobel/np.max(sobel))
    # 7) Create a binary mask where mag thresholds are met
    thresh_min = mag_thresh[0]
    thresh_max = mag_thresh[1]
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    
    combined_binary = np.zeros_like(sbinary)
    combined_binary[((binary_B == 1) & (binary_S == 1))|(binary_L == 1)] = 1
    
    # 8) Return a binary image of threshold result
    binary_output = np.copy(combined_binary) # placeholder line
    #binary_output = np.copy(sbinary) # placeholder line
    return binary_output
    
combine_binary = combine_select(undistorted, Sthresh=(90, 150), mag_thresh=(70, 255))
#result_path = '/Users/jianshuwang/Desktop/CarND-Advanced-Lane-Lines/output_images/test_out.jpg'

#cv2.imwrite(result_path,combine_binary)
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Undistorted Image', fontsize=50)
ax2.imshow(combine_binary, cmap='gray')
ax2.set_title('Combined Filter Binary Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.savefig('output_images/gradient_binary_result.png')



## 3. Apply a Perspective Transform to Rectify Binary Image

In [ ]:
point_left_up = (580, 460)
point_left_down = (280, 670)
point_right_up = (730, 460)
point_right_down = (1150, 670)

src = np.float32([point_left_up, point_left_down, point_right_down, point_right_up])
dst = np.float32([(100, 0), (100, 720), (1150, 720), (1150, 0)])

M = cv2.getPerspectiveTransform(src, dst)   
img_size = (combine_binary.shape[1],combine_binary.shape[0])
   
            
#delete the next two lines
undistorted_drawlines = np.copy(undistorted) 
    
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()


cv2.line(undistorted_drawlines,point_left_up, point_left_down, (255,0,0), 2)
cv2.line(undistorted_drawlines,point_right_down, point_right_up, (255,0,0), 2)
cv2.line(undistorted_drawlines,point_left_up, point_right_up, (255,0,0), 2)


warped = cv2.warpPerspective(undistorted_drawlines, M, img_size, flags=cv2.INTER_LINEAR)

ax1.imshow(undistorted_drawlines)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(warped)
ax2.set_title('Perspective Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/perspective_result.png')

In [ ]:
#delete the next two lines
combine_binary_drawlines = np.copy(combine_binary) 
    
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()


#cv2.line(combine_binary_drawlines,point_left_up, point_left_down, (255,0,0), 2)
#cv2.line(combine_binary_drawlines,point_right_down, point_right_up, (255,0,0), 2)
#cv2.line(combine_binary_drawlines,point_left_up, point_right_up, (255,0,0), 2)


warped_binary = cv2.warpPerspective(combine_binary, M, img_size, flags=cv2.INTER_LINEAR)

ax1.imshow(combine_binary_drawlines)
ax1.set_title('Original Binary Image', fontsize=50)
ax2.imshow(warped_binary)
ax2.set_title('Perspective Binary Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/perspective_result2.png')

## 4. Detect Lane Pixels and Fit to Find The Lane Boundary.

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
     
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        ### (`right` or `leftx_current`) on their mean position ###

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
                              ploty])))])
    
    line_pts = np.hstack((line_window1, line_window2))
    window_img = np.zeros_like(out_img)
     # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([line_pts]), (0,255, 0))
    
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    
    center_of_lane = (leftx[0]+rightx)/2
    

    return result, center_of_lane[0]


out_img, center_of_lane = fit_polynomial(warped_binary)
print(center_of_lane)

plt.imshow(out_img)
plt.savefig('output_images/Lane_Pixel_Result.png')

## 5. Determine The Curvature of the Lane and Vehicle Position with Respect to Center.

In [ ]:
def fit_polynomial_curve(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    return ploty, left_fit, right_fit


def measure_curvature_pixels(binary_warped):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    ploty, left_fit, right_fit = fit_polynomial_curve(binary_warped)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    
    
    
    return left_curverad, right_curverad



# Calculate the radius of curvature in pixels for both lane lines
left_curverad, right_curverad = measure_curvature_pixels(warped_binary)


print(left_curverad, 'm', right_curverad, 'm')

## 6. Warp the Detected Lane Boundaries Back onto The Original Image.

In [ ]:
Minv = cv2.getPerspectiveTransform(dst, src)
wrap_img = cv2.warpPerspective(out_img, Minv, img_size, flags=cv2.INTER_LINEAR)
wrap_result = cv2.addWeighted(undistorted, 1, wrap_img, 1, 0)
plt.imshow(wrap_result)
plt.savefig('output_images/Example_Result.png')

## 7. Output Visual Display of the Lane Boundaries and Numerical Estimation of Lane Curvature and Vehicle Position.

In [ ]:
result = cv2.addWeighted(undistorted,1,wrap_img,1,0) 
offset = 25
side_pos = " Right "
cv2.putText(result,'Vehicle is' + str(round(offset,3))+'m'+side_pos+'of center',
            (50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2)
cv2.putText(result,'Radius of curvature:'+str(round((left_curverad + right_curverad)/2))+'m',
            (50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2) 

plt.imshow(result)

## 8. Now we build the pipeline and output processed image and save into the output_image folder

In [ ]:
def pipeline(img):
    undistorted = cal_undistort(img, objpoints, imgpoints)
    combine_binary = hls_select(undistorted, Sthresh=(140, 255), mag_thresh=(70, 255))
    M = cv2.getPerspectiveTransform(src, dst)   
    img_size = (combine_binary.shape[1],combine_binary.shape[0])
    warped = cv2.warpPerspective(combine_binary, M, img_size, flags=cv2.INTER_LINEAR)   

    out_img, center_of_lane = fit_polynomial(warped)
    left_curverad, right_curverad = measure_curvature_pixels(warped)
    
    Minv = cv2.getPerspectiveTransform(dst, src)
    wrap_img = cv2.warpPerspective(out_img, Minv, img_size, flags=cv2.INTER_LINEAR)
    result = cv2.addWeighted(undistorted,1,wrap_img,1,0) 
    
    xm_per_pix = xm_per_pix = 3.7/700
    offset = (img_size[0]/2 - center_of_lane)*xm_per_pix
 

    side_pos = 'right'
    if offset <0:
        side_pos = 'left'
    
    cv2.putText(result,'Vehicle is ' + str(round(abs(offset),3))+'m '+side_pos+' of center',
            (50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2)
    cv2.putText(result,'Radius of curvature: '+str(round((left_curverad + right_curverad)/2))+'m',
            (50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2) 
    return result


img = cv2.imread('/Users/jianshuwang/Desktop/CarND-Advanced-Lane-Lines/test_images/test5.jpg')
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
xxx = pipeline(img)
plt.imshow(xxx)
    
xxxx = cv2.cvtColor(xxx, cv2.COLOR_RGB2BGR)
cv2.imwrite('output_images/straight_lines2_out.jpg',xxxx)



## 9. Use the pipeline to process the project_video.mp4 and save the output into the output image folder
From the output video, this algorithms and parameters can do a very good job for the source video.

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = pipeline(image)

    return result

white_output = 'output_images/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

## 10. Use the pipeline to process the challenge_video.mp4 and save the output into the output image folder
My algorithms does not do a good job for the 2 challenge videos, I think I just need to adjust parameters.

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = pipeline(image)

    return result

white_output = 'output_images/challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

## 11. Use the pipeline to process the challenge_video.mp4 and save the output into the output image folder


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = pipeline(image)

    return result

white_output = 'output_images/harder_challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)